# Azure AutoML

Stworzony model mógłby posłużyć do oceny czekolany na podstawie kilku danych, dzięki czemu nie musielibyśmy kupować czekolany niskiej jakości.

### Dane

Do wykonania eksperymentu użyty został zbiór danych "Chocolate Bar Ratings" pobrany z serwisu [Kaggle](https://www.kaggle.com/rtatman/chocolate-bar-ratings). Przed wykonaniem eksperymentu zbiór danych został dodany w usłudze [Azure Machine Learning](https://ml.azure.com). Użyty został model regresji dla kolumny _Rating_. Wybrałem model regresji ponieważ niektóre wartości nie znajdują się w bazie jak np 1.25, 2.0, a wartości te mogłyby być użyte. Aby porównać wartości z wartościami faktycznymi na końcu utworzyłem funkcję, która zebrane wartości zaokrągląga podobnie do wartości kolumny _Rating_ do najbliższej wartości 0.25. Na końcu znajduje się porównanie wyników zarówno bez obliczenia tego jak i po.

### Informacje

W ćwiuczeniu znajdują się proste do zrozumienia opisy przed fragmentami kodu oraz techniczne komentarze, które jeśli to wymagane dokładnie opisują cel kodu.

# Kroki

### Sprawdzenie czy moduł załadował się poprawnie

In [2]:
import azureml.core as aml

# Wypisz wersję używanego SDK AzureML
print("SDK version:", aml.VERSION)

SDK version: 1.20.0


### Pobieranie workspace'u

In [3]:
# Pobierz workspace z pliku konfiguracyjnego, w trakcie wykonywania tego kodu, może być wymagane zalogowanie się do konta Azure
ws = aml.Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DWANXHB9Q to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


### Pobieranie i wyświetlanie użytego zbioru danych oraz podstawowych własności zbioru

In [4]:
# Pobieranie zbioru danych z workspace'u, kod pochodzi z zakładki Consume w stworzonym zbiorze
aml_dataset = aml.Dataset.get_by_name(ws, name='chocolate-ratings')

# Zamiana zbioru na łatwiejszy do obsługiwania panadas
full_df = aml_dataset.to_pandas_dataframe()
# Wyświetlenie pierwszych wierszy zbioru
full_df.head(5)

,Company,Specific Bean Origin,REF,Review Date,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin,Column12
0,A. Morin,Agua Grande,1876.0,2016.0,63.0,France,3.75,,Sao Tome,None
1,A. Morin,Kpime,1676.0,2015.0,70.0,France,2.75,,Togo,None
2,A. Morin,Atsane,1676.0,2015.0,70.0,France,3.00,,Togo,None
3,A. Morin,Akata,1680.0,2015.0,70.0,France,3.50,,Togo,None
4,A. Morin,Quilla,1704.0,2015.0,70.0,France,3.50,,Peru,None


In [5]:
# Wyświetl podstawowe statystyczne własności zbioru
full_df.describe()

,REF,Review Date,Cocoa Percent,Rating
count,1793.000000,1794.000000,1788.000000,1794.000000
mean,1035.508645,2011.977146,73.521812,3.225753
std,553.067457,14.958468,55.873409,1.738851
min,5.000000,1391.000000,42.000000,1.000000
25%,576.000000,2010.000000,70.000000,3.000000
50%,1069.000000,2013.000000,70.000000,3.250000
75%,1502.000000,2015.000000,75.000000,3.500000
max,1952.000000,2017.000000,2014.000000,74.000000


### Usunięcie danych, które nie wpłyną, a nawet mogą zaburzyć ocenę

In [7]:
# Kolumna REF jest usuwana, ponieważ jest to tylko identyfikator wpisu
# Kolumna Review Date jest usuwana, poniważ mimo iż wysokie oceny mogłby
# być wstawiane częściej lub rzadziej w miarę czasu i model mógłby się lepiej nauczyć,
# nie wpłynęło by to na lepszy dobór dla samych znaczących cech
aml_dataset = aml_dataset.drop_columns(['REF', 'Review Date'])

full_df = aml_dataset.to_pandas_dataframe()

### Podział zbioru na dane treningowe i testowe

Pozwoli to ocenić skuteczność klasyfikatora 

In [8]:
# Podział zbioru na dane testowe i dane treningowe
train_dataset, test_dataset = aml_dataset.random_split(0.7, seed=1337)

### Wypisanie dostępnych jednostek obliczeniowych

In [16]:
from azureml.core.compute import ComputeTarget

# Wypisanie dostępnych jednostek obliczeniowych
ComputeTarget.list(ws)

[AmlCompute(workspace=Workspace.create(name='aml', subscription_id='416600d1-89d9-4ee5-9a25-a6ca53e3c797', resource_group='aml'), name=clusteraml, id=/subscriptions/416600d1-89d9-4ee5-9a25-a6ca53e3c797/resourceGroups/aml/providers/Microsoft.MachineLearningServices/workspaces/aml/computes/clusteraml, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None),
 {
   "id": "/subscriptions/416600d1-89d9-4ee5-9a25-a6ca53e3c797/resourceGroups/aml/providers/Microsoft.MachineLearningServices/workspaces/aml/computes/instanceaml",
   "name": "instanceaml",
   "location": "westeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "westeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS2_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https

### Znalezienie maszyny obliczeniowej do wykonania doświadczenia

Do znalezienia maszyny używana jest jej nazwa znaleziona w poprzednim zadaniu.

Czemu musimy ją znaleźć a nie mówimy, że ją mamy? To trochę tak, jak z rozmową przez telefon, wiemy jak się ktoś nazywa, ale jeśli chcemy z nim porozmawiać musimy najpierw zadzwonić.

In [17]:
# Wpisujemy jedną z wartości która została znaleziona w poprzednim zadaniu, musi być to wartość z typem AmlCompute
amlcompute_cluster_name = "clusteraml"

found = False

# Sprawdzamy czy nie było żadnej pomyłki w nazwie, czy typ się zgadza itp.

cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing training cluster.')
    # Pobieranie klastra obliczeniowego
    aml_remote_compute = cts[amlcompute_cluster_name]
        
    print('Checking cluster status...')
    # Sprawdzanie statusu i skalowanie klastra 
    aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

    # Wypisanie dodatkowych informacji o klastrze
    print(aml_remote_compute.get_status().serialize())
    
if not found:
    # Jeśli zostanie wypisane to nie należy iść do kolejnego kroku
    print('Training cluster not found...')


Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-26T22:42:16.153000+00:00', 'errors': None, 'creationTime': '2021-01-26T22:00:10.733210+00:00', 'modifiedTime': '2021-01-26T22:00:27.278900+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


### Tworzone są ustawienia klasyfikacji

Dane będą przewidywane liniowo, tzn. ocena póki co nie będzie co 0.25 jak to ma miejsce w danych tylko będzie przewidywana jako liczba w wartościach od 1 do 5 z możliwością np 1.352214 itp.

In [18]:
from azureml.train import automl

# #Pobieranie listy możliwych metryk regresji
automl.utilities.get_primary_metrics('regression')

['normalized_root_mean_squared_error',
 'spearman_correlation',
 'normalized_mean_absolute_error',
 'r2_score']

In [19]:
import logging
import os

from azureml.train.automl import AutoMLConfig

# Tworzenie folderu gdzie będą zapisywane logi
project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

# Konfigurowanie eksperymentu, używana jest metryka NRMSE i jako kolumnę która będzie przewidywana używana jest kolumna "Rating"
# Używany jest tylko zbiór treningowy
automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="Rating",
                             n_cross_validations=5,                                               
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

### Tworzenie funkcji regresji

Tworzona jest funkcja dzięki której mozna przewidzieć wartość wyniku - pola _Rating_.  

In [20]:
from datetime import datetime

# Tworzenie w miarę unikalnej nazwy - przynajmniej w tym projekcie monieważ pobierana jest data
now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "classif-automl-remote-{0}".format(time_string)
print(experiment_name)

# Tworzenie eksperymentu
experiment = aml.Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()

# Wysyłanie eksperymentu - czyli tworzenie funkcji regresji
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

classif-automl-remote-01-27-2021-00
Running on remote.
No run_configuration provided, running on clusteraml with default configuration
Running on remote compute: clusteraml
Parent Run ID: AutoML_c27f743e-39c2-42bb-bf52-aabe314579ab

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS

In [21]:
# Wyświetlanie danych za pomocą bibliote
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Wyświetlanie informacji o modelu - funkcji, która najlepiej dopasowuje się do danych

In [22]:
# Pobieranie i wyświetlanie wyniku i informacji o utworzonym modelu
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: classif-automl-remote-01-27-2021-00,
Id: AutoML_c27f743e-39c2-42bb-bf52-aabe314579ab_15,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                          

### Sprawdzanie poprawności stworzonego modelu

Po utworzeniu modelu, należy sprwadzić jak dobrze model potrafi przewidzieć wartości z danych, które wcześniej zostały oddzielone od danych treningowych. Jak poprzednio wspomniano, funkcja zwraca wartości, które nie są zaokrąglone do najbliższej wartości 0.25, dlatego sstworzona została funkcja która to robi.

In [76]:
test_dataset_df = test_dataset.to_pandas_dataframe()

# Pobierani wartości wejściowych i wyjściowych z danych testowych
if 'Rating' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('Rating')

x_test_df = test_dataset_df

In [77]:
# Przewidywanie wartości testowych
y_predictions = fitted_model.predict(x_test_df)

print('10 predykcji bez zaokrąglania:')
print(y_predictions[15:25])

# Aby zwiększyć precyzję modelu, a wyniki bardziej odpowiadały wartościom,
# które powinny być tam stworzone wyżej stworzyłem funkcję, która zaokrągla wynik
# do najlbliższej wartości 0.25.
def roundToNearest25(a):
    return (a * 4).round() / 4

print('10 predykcji po zaokrągleniu:')
print(roundToNearest25(y_predictions[15:25]))

10 predykcji bez zaokrąglania:
[3.06514215 3.06514215 3.10084701 3.06514215 2.93975115 3.21346474
 3.13048649 3.17015243 3.70869875 3.79058003]
10 predykcji po zaokrągleniu:
[3.   3.   3.   3.   3.   3.25 3.25 3.25 3.75 3.75]


In [80]:
print('Precyzja (accuracy) modelu bez zaokrąglania:')
print(fitted_model.score(x_test_df, y_test_df))

print('Precyzja (accuracy) modelu z zaokrąglaniem:')
print(fitted_model.score(x_test_df, roundToNearest25(y_predictions)))

Precyzja (accuracy) modelu bez zaokrąglania:
0.1570196313983816
Precyzja (accuracy) modelu z zaokrąglaniem:
0.8669471902873664


### Wnioski

Taki wynik sprawia, że model nie działa świetnie, ale wyniki przez niego zwracane są bliższe wartościom faktycznym, niż gdyby użyta została funkcja losująca wynik.